In [36]:
import torch
from taskdataset import TaskDataset
import re
import pandas as pd
import matplotlib.pyplot as plt
dataset = torch.load("data/ModelStealingPub.pt")
#print(dataset.ids, dataset.imgs, dataset.labels)

In [71]:

from typing import List
from dotenv import load_dotenv


import numpy as np
import requests
import os

load_dotenv()

SERVER_URL = str("http://34.71.138.79:9090/")
TEAM_TOKEN = str("juU0vkSeNhJdtzj7")




In [38]:
df_images = pd.DataFrame(list(dataset), columns = ['id', 'img','Target'])

In [39]:
df_images#['img'][0]

,id,img,Target
0,73838,<PIL.Image.Image image mode=RGB size=32x32 at ...,40019202
1,59663,<PIL.Image.Image image mode=RGB size=32x32 at ...,10964686
2,288406,<PIL.Image.Image image mode=RGB size=32x32 at ...,62225416
3,173427,<PIL.Image.Image image mode=RGB size=32x32 at ...,26466207
4,188902,<PIL.Image.Image image mode=RGB size=32x32 at ...,24683694
...,...,...,...
12995,61183,<PIL.Image.Image image mode=RGB size=32x32 at ...,50011542
12996,268157,<PIL.Image.Image image mode=RGB size=32x32 at ...,62225416
12997,74671,<PIL.Image.Image image mode=RGB size=32x32 at ...,75652382
12998,22424,<PIL.Image.Image image mode=RGB size=32x32 at ...,70483417


In [40]:
def extract_mode(text):
    # Define a regular expression pattern to match "mode=" followed by any non-space characters
    pattern = r'mode=([^ ]+)'
    
    # Search for the pattern in the text
    match = re.search(pattern, text)
    
    # If a match is found, return the extracted mode, otherwise return None
    if match:
        return match.group(1)
    else:
        return None

# Example text
text = 'This is a sample text with mode=123 and some additional information.'

# Extract mode from the text
mode = extract_mode(text)

In [41]:
list_col= []
for i in range(13000):
    list_col.append(extract_mode(str(dataset[i][1])))

In [42]:
df = pd.DataFrame({'ID': dataset[:][0],'RGB': list_col, 'Target':dataset[:][2] })
df.sort_values("ID", inplace = True)
df = df.set_index("ID")

In [43]:
df_color = df[df["RGB"] == "RGB"]

In [44]:
top10 = df_color["Target"].value_counts().head(10).index

In [45]:
df_final = df_color[df_color['Target'].isin(top10)]

In [46]:
df_final['Target'] = pd.Categorical(df_final['Target'], categories=top10, ordered=True)

# Sort the DataFrame based on the categorical column 'B'
df_sorted = df_final.sort_values(by='Target')


C:\Users\Admin\AppData\Local\Temp\ipykernel_22400\1911434496.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['Target'] = pd.Categorical(df_final['Target'], categories=top10, ordered=True)


In [47]:
df_sorted.head()#.index

,RGB,Target
ID,,
185611,RGB,62225416
216847,RGB,62225416
216895,RGB,62225416
216921,RGB,62225416
216932,RGB,62225416


In [67]:
def model_stealing(path_to_png_file: str):
    endpoint = "/modelstealing"
    url = SERVER_URL + endpoint
    with open(path_to_png_file, "rb") as f:
        response = requests.get(url, files={"file": f}, headers={"token": TEAM_TOKEN})
        if response.status_code == 200:
            representation = response.json()["representation"]
            print("Request ok")
            print(representation)
        else:
            raise Exception(
                f"Model stealing failed. Code: {response.status_code}, content: {response.json()}"
            )

In [49]:
df_images = df_images.set_index('id')

In [50]:
img  = df_images.loc[df_sorted.index[0],'img']
image_path_bench = 'picture_benchmark.png' 
img.save(image_path_bench)

In [72]:
list_input = []
list_output = [] 

lista_mse = []

for i,j in zip(df_sorted.index, range(df_sorted.shape[0])):
    print(i)
    print(j)
    img  = df_images.loc[i,'img']
    image_path = 'picture.png'  # Specify the path and file name where you want to save the image
    # Save the image
    img.save(image_path)
    if j ==0:
        value_bench = model_stealing(image_path)
    display(img)
    if (j)%300 ==0:
        value = model_stealing(image_path_bench)
        list_input.append(df_sorted.index[0])
        lista_output.append(value_bench)
        lista_mse.append(np.mean((value - value_bench) ** 2))
    list_input.append(i)
    lista_output.append(model_stealing(image_path))
    break

185611
0


ConnectTimeout: HTTPConnectionPool(host='34.71.138.79', port=9090): Max retries exceeded with url: //modelstealing (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x000001F0B9FFEBC0>, 'Connection to 34.71.138.79 timed out. (connect timeout=None)'))

# test dla jednego